In [1]:
from girder_client import GirderClient
import pandas as pd
import birdshot
import os
from birdshot import summarize_presence_by_sample

In [2]:
# Authenticate with GirderClient
client = GirderClient(apiUrl=os.environ['GIRDER_API_URL'])
client.authenticate(apiKey=os.environ["GIRDER_API_KEY"])

{'_id': '6424afee4236ff9b0883f24b'}

In [3]:
campaign = 'CBA'
df = birdshot.query(campaign=campaign, client=client)
summary_df = summarize_presence_by_sample(df, campaign)
summary_df.to_csv(f"data/{campaign}_completion_summary.csv", index=False)